1. Przygotowanie danych

In [1]:
import time

from sklearn.datasets import fetch_openml
import numpy as np
mnist=fetch_openml('mnist_784', version=1)

print((np.array(mnist.data.loc[42]).reshape(28,28)>0).astype(int))

C:\Users\asiaw\anaconda3\envs\UM\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 

2. Zbiór uczący i testowy

In [2]:
X, y = mnist["data"], mnist["target"].astype(np.uint8)
print(X.shape, y.shape)

y = sorted(y)
X.reindex()

X_train, X_test = X[:56000], X[56000:]
y_train, y_test = y[:56000], y[56000:]

print(set(y_train))
print(set(y_test))

(70000, 784) (70000,)
{0, 1, 2, 3, 4, 5, 6, 7}
{8, 9, 7}


In [3]:
from sklearn.model_selection import train_test_split
X, y = mnist["data"], mnist["target"].astype(np.uint8)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

3. Uczenie, jedna klasa

In [4]:
from sklearn.linear_model import SGDClassifier
y_train_0 = (y_train == 0)
y_test_0 = (y_test == 0)

#start = time.time()
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_0)
#print(time.time() - start)

SGDClassifier(random_state=42)

In [5]:
from sklearn.model_selection import cross_val_score

cva = cross_val_score(sgd_clf, X_train, y_train_0, scoring="accuracy", n_jobs=-1, cv=3)

acc_train = sgd_clf.score(X_train, y_train_0)
acc_test = sgd_clf.score(X_test, y_test_0)
print(acc_test)
print(acc_train)

0.9903571428571428
0.9918571428571429


In [6]:
import pickle
with open("sgd_cva.pkl", 'wb') as f:
    pickle.dump(cva, f)

with open("sgd_acc.pkl", 'wb') as f:
    pickle.dump([acc_train, acc_test], f)

5. Uczenie, wiele klas

In [7]:
svm_clf = SGDClassifier()
svm_clf.fit(X_train, y_train)

SGDClassifier()

In [8]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_train_pred = cross_val_predict(svm_clf, X_train, y_train, cv=3, n_jobs=-1)
conf_mx = confusion_matrix(y_train, y_train_pred)
print(conf_mx)

[[5177    0   89   26    7   90   54   20   65   32]
 [   1 5917   51   44    4   23    9   17  205    6]
 [  30   98 4781  234   44   33   84  103  176   27]
 [  31   36  184 4910    9  183   21  119  131   84]
 [  12   37   52   33 4769   44   39   80  128  335]
 [  70   29   65  355   57 3554  151   37  643   79]
 [  37   26   73   20   19  139 5084   11   69    2]
 [  19   26   66   43   37   17    4 5296   64  218]
 [  40  200  151  202   20  316   37   79 4280  143]
 [  14   20   23   96  116   67    2  455  168 4577]]


In [9]:
with open("sgd_cmx.pkl", 'wb') as f:
    pickle.dump(conf_mx, f)